### Обучение YOLOv8 для сегментации дорожной разметки

### Классы датасета
- Bus Lane (BL)	(0,255,255)
- Cycle Lane (CL)	(0,128,255) 
- Diamond (DM)	(178,102,255)
- Junction Box (JB)	(255,255,51)
- Left Arrow (LA)	(255,102,178)
- Pedestrian Crossing (PC)	(255,255,0)
- Right Arrow (RA)	(255,0,127)
- Straight Arrow (SA)	(255,0,255)
- Slow (SL)	(0,255,0)
- Straight-Left Arrow (SLA)	(255,128,0)
- Straight-Right Arrow (SRA)	(255,0,0)


In [1]:
import os

os.makedirs('/kaggle/working/polygon_txt', exist_ok = True)
os.makedirs('/kaggle/working/train_txt', exist_ok = True)
os.makedirs('/kaggle/working/train_images', exist_ok = True)
os.makedirs('/kaggle/working/val_images', exist_ok = True)
os.makedirs('/kaggle/working/val_txt', exist_ok = True)

In [2]:
import os
import json
import shutil
dirname = "/kaggle/input/roadmarking/RoadMarking/RoadMarking/1/train/train/polygon_annotations"
train_txt = '/kaggle/working/polygon_txt'
labels_to_number ={
    "DM":0,
    "PC":1,
    "SA":2
}
for root, dirs, files in os.walk(dirname):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            
        file_name = os.path.splitext(file)[0]
        
        image_width = data['imageWidth']
        image_height = data['imageHeight']
        shapes = data["shapes"]
        
        output_lines = []
        for shape in shapes:
            label = shape['label']
            if label not in labels_to_number:
                continue
            normalized_points = []
            for x, y in shape['points']:
                nx = x / image_width
                ny = y / image_height
                normalized_points.extend([nx, ny])
            class_id = labels_to_number[label]
            line = f"{class_id} " + " ".join(map(str, normalized_points))
            output_lines.append(line)
        
        output_file = f"{file_name}.txt"
        with open(output_file, "w") as f:
            f.write("\n".join(output_lines))
        shutil.move(output_file, f'{train_txt}/{output_file}')

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

annotations = []

label_dir = "/kaggle/working/polygon_txt" 

annotations = []
for txt_file in os.listdir(label_dir):
    with open(f"{label_dir}/{txt_file}") as f:
        for line in f:
            class_id = int(line.split()[0])
            annotations.append({"file": txt_file, "class_id": class_id})

df = pd.DataFrame(annotations)
print(df["class_id"].value_counts())

class_id
2    1088
0     770
1     611
Name: count, dtype: int64


Наблюдаем сильный дисбаланс классов, попробуем  стратифицированное разбиение

In [4]:
train_files, val_files = train_test_split(
    df["file"].unique(),
    test_size = 0.2,
    stratify = df.groupby("file")["class_id"].first(),
    random_state = 42
)

In [5]:
image_dir = "/kaggle/input/roadmarking/RoadMarking/RoadMarking/1/train/train/images"
os.makedirs('/kaggle/working/dataset', exist_ok = True)
os.makedirs('/kaggle/working/dataset/images/train', exist_ok = True)
os.makedirs('/kaggle/working/dataset/images/val', exist_ok = True)
os.makedirs('/kaggle/working/dataset/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/dataset/labels/val', exist_ok = True)
for file in train_files:
    # Копируем изображения
    shutil.copy(f"{image_dir}/{file.replace('.txt', '.jpg')}", "/kaggle/working/dataset/images/train")
    # Копируем аннотации
    shutil.copy(f"{label_dir}/{file}", "/kaggle/working/dataset/labels/train")

for file in val_files:
    shutil.copy(f"{image_dir}/{file.replace('.txt', '.jpg')}", "/kaggle/working/dataset/images/val")
    shutil.copy(f"{label_dir}/{file}", "/kaggle/working/dataset/labels/val")

print("Разбиение завершено!")
print(f"Train: {len(train_files)} файлов, Val: {len(val_files)} файлов")

Разбиение завершено!
Train: 1378 файлов, Val: 345 файлов


In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing ins

In [7]:
from ultralytics import YOLO
model = YOLO("yolov8n-seg.pt")
results = model.train(
    data = "/kaggle/input/roadmarking/data.yaml",
    epochs = 30,
    imgsz = 640,
    batch = 16,
    name='yolov8n_seg_custom'
)
metrics = model.val()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.11 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/roadmarking/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        345        496       0.96      0.949      0.977      0.776      0.946      0.924      0.959      0.663
                    DM         93        163       0.97      0.989      0.992       0.78      0.943      0.951      0.968      0.573
                    PC        124        124       0.97      0.984      0.986      0.784      0.972      0.984      0.986      0.784
                    SA        129        209      0.941      0.876      0.954      0.765      0.923      0.837      0.923      0.633
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /kaggle/working/runs/segment/yolov8n_seg_custom
Ultralytics 8.4.11 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 86 layers, 3,258,649 parameters, 0 gradients, 11.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3310.4±712.3 MB/s, size: 555.9 KB)
val: Scanning /kaggle/working/dataset/labels/val.cache.

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        345        496      0.961      0.952      0.983      0.775      0.944      0.925      0.959      0.651
                    DM         93        163      0.976      0.991      0.992      0.781      0.939      0.947      0.967      0.549
                    PC        124        124      0.969      0.984      0.986      0.777      0.971      0.984      0.986      0.779
                    SA        129        209      0.939       0.88       0.97      0.768      0.922      0.845      0.923      0.624
Speed: 1.6ms preprocess, 2.8ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /kaggle/working/runs/segment/val


In [17]:
print(metrics.summary())


[{'Class': 'DM', 'Images': 93, 'Instances': 163, 'Box-P': 0.97582, 'Box-R': 0.99055, 'Box-F1': 0.98313, 'mAP50': 0.99226, 'mAP50-95': 0.78068, 'Mask-P': 0.93917, 'Mask-R': 0.94725, 'Mask-F1': 0.94319}, {'Class': 'PC', 'Images': 124, 'Instances': 124, 'Box-P': 0.96919, 'Box-R': 0.98387, 'Box-F1': 0.97648, 'mAP50': 0.98648, 'mAP50-95': 0.77664, 'Mask-P': 0.97125, 'Mask-R': 0.98387, 'Mask-F1': 0.97752}, {'Class': 'SA', 'Images': 129, 'Instances': 209, 'Box-P': 0.93933, 'Box-R': 0.88038, 'Box-F1': 0.9089, 'mAP50': 0.97041, 'mAP50-95': 0.76779, 'Mask-P': 0.9217, 'Mask-R': 0.84484, 'Mask-F1': 0.8816}]


Все метрики > 0.88 — модель работает очень хорошо

Маски сегментации почти не уступают боксам (разница < 0.06) — сегментация точная
1. DM
Лучший класс по mAP50-95 (0.781) — лучше всех детектирует с разными порогами IoU

Самый высокий Recall (0.99055) — находит практически все объекты

2. PC
Самый высокий Mask-F1 (0.978) — лучший баланс между точностью и полнотой сегментации

Почти идеальная сегментация (Mask-P: 0.971, Mask-R: 0.984)

3. SA
Наиболее сложный класс — самые низкие метрики, но всё ещё высокие

Recall сегментации (0.845) заметно ниже